In [1]:
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent))

from datetime import date, timedelta
import codecs
import pandas as pd

pd.set_option('display.max_columns', None)

from os.path import join

from bulletin import default_input, default_output
from bulletin.services.metabase import Metabase
from bulletin.casos_confirmados import CasosConfirmados
from bulletin.utils.static import Municipios
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash, normalize_number
from bulletin.utils import static
from bulletin.utils.xls_ import fit_cols

from datetime import datetime


---------------------------------------------------------------

root: C:\SESA\covid19datascience\bulletin
parent: C:\SESA\covid19datascience
default_input: C:\SESA\covid19datascience\input
default_output: C:\SESA\covid19datascience\output

---------------------------------------------------------------



In [2]:
cc = CasosConfirmados()

databases: ['cc_27_05_2021', 'cc_28_05_2021', 'cc_29_05_2021', 'cc_30_05_2021', 'cc_31_05_2021']


In [3]:
cc.load()

Load C:\SESA\covid19datascience\bulletin\database\casos_confirmados\cc_31_05_2021.pkl
loading Casos Confirmados from pkl time elepsed:00:00:58


In [4]:
casos = cc.df

In [5]:
obitos_curitiba = casos.loc[(casos['ibge_resid']==410690) & (casos['evolucao']==2)].sort_values('nome').copy()
obitos_curitiba['hash_obito'] = obitos_curitiba['nome'].apply(normalize_hash) + obitos_curitiba['data_evolucao'].apply(date_hash)

obitos_curitiba

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_atend,hash_less,hash_more,hash_diag,hash_obito
860283,945449,-1.0,PR,410690,410690,ABEGAIL JOSEFA MILLER,F,63,"COVID-19, Biologia Molecular",2021-04-04,2021-04-06 00:00:00,2021-03-25 00:00:00,2.0,2021-04-04 00:00:00,2021-04-20,ABEGAILJOSEFAMILLER63410690,ABEGAILJOSEFAMILLER63410690,ABEGAILJOSEFAMILLER62410690,ABEGAILJOSEFAMILLER64410690,ABEGAILJOSEFAMILLER04042021,ABEGAILJOSEFAMILLER04042021
769481,851466,-1.0,PR,410690,410690,ABEL ARMANDO DA RESSURREICAO,M,75,"COVID-19, Biologia Molecular",2021-03-15,2021-03-17 00:00:00,2021-03-07 00:00:00,2.0,2021-03-29 00:00:00,2021-04-01,ABELARMANDODARESSURREICAO75410690,ABELARMANDODARESSURREICAO75410690,ABELARMANDODARESSURREICAO74410690,ABELARMANDODARESSURREICAO76410690,ABELARMANDODARESSURREICAO15032021,ABELARMANDODARESSURREICAO29032021
851205,936365,-1.0,PR,410690,410690,ABEL DOS SANTOS,M,58,"COVID-19, Biologia Molecular",2021-03-31,2021-04-02 00:00:00,2021-03-19 00:00:00,2.0,2021-04-06 00:00:00,2021-04-10,ABELDOSSANTOS58410690,ABELDOSSANTOS58410690,ABELDOSSANTOS57410690,ABELDOSSANTOS59410690,ABELDOSSANTOS31032021,ABELDOSSANTOS06042021
873824,958999,-1.0,PR,410690,410690,ABENER DONIZETE MENDES,M,56,Vírus Respiratório,2021-03-15,2021-04-09 00:00:00,NaN,2.0,2021-03-17 00:00:00,2021-04-09,ABENERDONIZETEMENDES56410690,ABENERDONIZETEMENDES56410690,ABENERDONIZETEMENDES55410690,ABENERDONIZETEMENDES57410690,ABENERDONIZETEMENDES15032021,ABENERDONIZETEMENDES17032021
680664,762629,-1.0,PR,410690,410690,ABI TEIXEIRA DOS SANTOS,F,66,"COVID-19, Biologia Molecular",2020-12-02,2021-03-07 00:00:00,NaN,2.0,2020-11-29 00:00:00,2021-03-07,ABITEIXEIRADOSSANTOS66410690,ABITEIXEIRADOSSANTOS66410690,ABITEIXEIRADOSSANTOS65410690,ABITEIXEIRADOSSANTOS67410690,ABITEIXEIRADOSSANTOS02122020,ABITEIXEIRADOSSANTOS29112020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823904,905908,-1.0,PR,410690,410690,ZULEMIA ANNA DO VALLE RIBEIRO,F,85,"COVID-19, Biologia Molecular",2021-03-19,2021-03-25 00:00:00,2021-03-15 00:00:00,2.0,2021-04-01 00:00:00,2021-04-02,ZULEMIAANNADOVALLERIBEIRO85410690,ZULEMIAANNADOVALLERIBEIRO85410690,ZULEMIAANNADOVALLERIBEIRO84410690,ZULEMIAANNADOVALLERIBEIRO86410690,ZULEMIAANNADOVALLERIBEIRO19032021,ZULEMIAANNADOVALLERIBEIRO01042021
855769,940931,-1.0,PR,410690,410690,ZULMEIA FIGUEIREDO,F,80,"COVID-19, Biologia Molecular",2021-03-10,2021-04-02 00:00:00,2021-03-03 00:00:00,2.0,2021-03-14 00:00:00,2021-04-02,ZULMEIAFIGUEIREDO80410690,ZULMEIAFIGUEIREDO80410690,ZULMEIAFIGUEIREDO79410690,ZULMEIAFIGUEIREDO81410690,ZULMEIAFIGUEIREDO10032021,ZULMEIAFIGUEIREDO14032021
855770,940932,-1.0,PR,410690,410690,ZULMIRA ALVES DE SOUZA,F,80,"COVID-19, Biologia Molecular",2021-03-12,2021-04-02 00:00:00,2021-03-05 00:00:00,2.0,2021-03-17 00:00:00,2021-04-02,ZULMIRAALVESDESOUZA80410690,ZULMIRAALVESDESOUZA80410690,ZULMIRAALVESDESOUZA79410690,ZULMIRAALVESDESOUZA81410690,ZULMIRAALVESDESOUZA12032021,ZULMIRAALVESDESOUZA17032021
483423,561366,-1.0,PR,410690,410765,ZULMIRA FARIAS BATTU,F,59,"COVID-19, Biologia Molecular",2021-01-07,2021-01-12 00:00:00,2021-01-02 00:00:00,2.0,2021-02-12 00:00:00,2021-02-28,ZULMIRAFARIASBATTU59410690,ZULMIRAFARIASBATTU59410765,ZULMIRAFARIASBATTU58410690,ZULMIRAFARIASBATTU60410690,ZULMIRAFARIASBATTU07012021,ZULMIRAFARIASBATTU12022021


In [7]:
curitiba = pd.read_excel(join(default_input,'curitiba.xlsx'))
curitiba.columns = [ normalize_labels(x) for x in curitiba.columns]

curitiba['hash'] = (curitiba['nome'].apply(normalize_hash) +
                    curitiba['idade'].astype(str) +
                    '410690' )

curitiba['hash_less'] = ( curitiba['nome'].apply(normalize_hash) +
                        curitiba['idade'].apply(lambda x: str(x - 1)) +
                        '410690' )

curitiba['hash_more'] = ( curitiba['nome'].apply(normalize_hash) +
                        curitiba['idade'].apply(lambda x: str(x + 1)) +
                        '410690' )

curitiba['hash_obito'] = curitiba['nome'].apply(normalize_hash) + curitiba['data_do_obito'].apply(date_hash)

curitiba = curitiba.sort_values('nome')
curitiba

,data_do_obito,data_da_divulgacao,nome,data_nasc,idade,sexo,confirmado,inicio_dos_sintomas,munic_resid,cpf,hash,hash_less,hash_more,hash_obito
4102,2021-04-04,2021-04-08,ABEGAIL JOSEFA MILLER,1957-12-27,63,F,S,2021-03-25,CURITIBA,NaN,ABEGAILJOSEFAMILLER63410690,ABEGAILJOSEFAMILLER62410690,ABEGAILJOSEFAMILLER64410690,ABEGAILJOSEFAMILLER04042021
3815,2021-03-29,2021-03-30,ABEL ARMANDO DA RESSURREICAO,1945-07-30,75,M,S,2021-03-06,CURITIBA,2.791448e+09,ABELARMANDODARESSURREICAO75410690,ABELARMANDODARESSURREICAO74410690,ABELARMANDODARESSURREICAO76410690,ABELARMANDODARESSURREICAO29032021
4060,2021-04-06,2021-04-07,ABEL DOS SANTOS,1962-11-08,58,M,S,2021-03-19,CURITIBA,4.518940e+10,ABELDOSSANTOS58410690,ABELDOSSANTOS57410690,ABELDOSSANTOS59410690,ABELDOSSANTOS06042021
3688,2021-03-16,2021-03-27,ABENER DONIZETE MENDES,1964-11-03,56,M,S,2021-03-09,CURITIBA,4.733923e+10,ABENERDONIZETEMENDES56410690,ABENERDONIZETEMENDES55410690,ABENERDONIZETEMENDES57410690,ABENERDONIZETEMENDES16032021
1775,2020-11-29,2020-12-03,ABI TEIXEIRA DOS SANTOS,1954-09-25,66,F,S,2020-11-29,CURITIBA,NaN,ABITEIXEIRADOSSANTOS66410690,ABITEIXEIRADOSSANTOS65410690,ABITEIXEIRADOSSANTOS67410690,ABITEIXEIRADOSSANTOS29112020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,2021-04-01,2021-04-03,ZULEMIA ANNA DO VALLE RIBEIRO,1936-02-12,85,F,S,2021-03-15,CURITIBA,7.208373e+10,ZULEMIAANNADOVALLERIBEIRO85410690,ZULEMIAANNADOVALLERIBEIRO84410690,ZULEMIAANNADOVALLERIBEIRO86410690,ZULEMIAANNADOVALLERIBEIRO01042021
3261,2021-03-14,2021-03-15,ZULMEIA FIGUEIREDO,1940-05-29,80,F,S,2021-03-03,CURITIBA,NaN,ZULMEIAFIGUEIREDO80410690,ZULMEIAFIGUEIREDO79410690,ZULMEIAFIGUEIREDO81410690,ZULMEIAFIGUEIREDO14032021
3687,2021-03-17,2021-03-26,ZULMIRA ALVES DE SOUZA,1941-01-06,80,F,S,2021-03-08,CURITIBA,3.851727e+09,ZULMIRAALVESDESOUZA80410690,ZULMIRAALVESDESOUZA79410690,ZULMIRAALVESDESOUZA81410690,ZULMIRAALVESDESOUZA17032021
2902,2021-02-12,2021-02-26,ZULMIRA FARIAS BATTU,1961-04-06,59,F,S,2021-01-01,CURITIBA,5.374200e+10,ZULMIRAFARIASBATTU59410690,ZULMIRAFARIASBATTU58410690,ZULMIRAFARIASBATTU60410690,ZULMIRAFARIASBATTU12022021


In [12]:
obitos_curitiba_nao_parana = curitiba.loc[~(
    (curitiba['hash'].isin(obitos_curitiba['hash'])) |
    (curitiba['hash'].isin(obitos_curitiba['hash_less'])) |
    (curitiba['hash'].isin(obitos_curitiba['hash_more'])) |
    (curitiba['hash_obito'].isin(obitos_curitiba['hash_obito']))
)]
obitos_curitiba_nao_parana.to_excel(join(default_output,'obitos_curitiba_nao_parana.xlsx'))
obitos_curitiba_nao_parana

,data_do_obito,data_da_divulgacao,nome,data_nasc,idade,sexo,confirmado,inicio_dos_sintomas,munic_resid,cpf,hash,hash_less,hash_more,hash_obito
5230,2021-05-23,2021-05-24,ABIGAIL SILVA RODRIGUES,1949-01-20,72,F,S,2021-05-11,CURITIBA,7.550173e+10,ABIGAILSILVARODRIGUES72410690,ABIGAILSILVARODRIGUES71410690,ABIGAILSILVARODRIGUES73410690,ABIGAILSILVARODRIGUES23052021
151,2020-07-01,2020-07-02,ADELVINO FRANCISCO GOMES,1930-06-08,90,M,S,2020-06-22,CURITIBA,NaN,ADELVINOFRANCISCOGOMES90410690,ADELVINOFRANCISCOGOMES89410690,ADELVINOFRANCISCOGOMES91410690,ADELVINOFRANCISCOGOMES01072020
4367,2021-04-15,2021-04-17,ADEMIR ROGERIO ROSA,1968-02-19,53,M,S,2021-03-12,CURITIBA,NaN,ADEMIRROGERIOROSA53410690,ADEMIRROGERIOROSA52410690,ADEMIRROGERIOROSA54410690,ADEMIRROGERIOROSA15042021
3898,2021-02-01,2021-04-02,ADILSON KARAM FRANCA,1939-11-22,81,M,S,2021-01-04,CURITIBA,8.717614e+09,ADILSONKARAMFRANCA81410690,ADILSONKARAMFRANCA80410690,ADILSONKARAMFRANCA82410690,ADILSONKARAMFRANCA01022021
4055,2021-04-06,2021-04-07,ADIRES LUIZ SOARES,1948-10-31,72,M,S,2021-03-20,CURITIBA,3.177488e+10,ADIRESLUIZSOARES72410690,ADIRESLUIZSOARES71410690,ADIRESLUIZSOARES73410690,ADIRESLUIZSOARES06042021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3461,2021-03-19,2021-03-20,YNOEMIO LUDVIG,1937-05-09,83,M,S,2021-02-23,CURITIBA,NaN,YNOEMIOLUDVIG83410690,YNOEMIOLUDVIG82410690,YNOEMIOLUDVIG84410690,YNOEMIOLUDVIG19032021
2175,2020-12-24,2020-12-28,YVONNE FELICISSIMA DE ANDRADE,1924-04-25,96,F,S,2020-12-18,CURITIBA,NaN,YVONNEFELICISSIMADEANDRADE96410690,YVONNEFELICISSIMADEANDRADE95410690,YVONNEFELICISSIMADEANDRADE97410690,YVONNEFELICISSIMADEANDRADE24122020
4668,2021-04-06,2021-04-30,ZENI CAVALHEIRO DA CRUZ FERREIRA,1946-03-26,74,F,S,2021-02-18,CURITIBA,5.901151e+09,ZENICAVALHEIRODACRUZFERREIRA74410690,ZENICAVALHEIRODACRUZFERREIRA73410690,ZENICAVALHEIRODACRUZFERREIRA75410690,ZENICAVALHEIRODACRUZFERREIRA06042021
2770,2021-02-12,2021-02-16,ZILA DO PRADO,1955-02-19,65,F,S,2021-01-18,CURITIBA,5.769922e+10,ZILADOPRADO65410690,ZILADOPRADO64410690,ZILADOPRADO66410690,ZILADOPRADO12022021


In [13]:
obitos_parana_nao_curitiba = obitos_curitiba.loc[~(
    (obitos_curitiba['hash'].isin(curitiba['hash'])) |
    (obitos_curitiba['hash'].isin(curitiba['hash_less'])) |
    (obitos_curitiba['hash'].isin(curitiba['hash_more'])) |
    (obitos_curitiba['hash_obito'].isin(curitiba['hash_obito']))
)]
obitos_parana_nao_curitiba.to_excel(join(default_output,'obitos_parana_nao_curitiba.xlsx'))
obitos_parana_nao_curitiba

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_atend,hash_less,hash_more,hash_diag,hash_obito
680721,762686,-1.0,PR,410690,410690,ADALBERTO RICARDO DOS SANTOS FI,M,93,"COVID-19, Biologia Molecular",2020-12-21,2021-03-07 00:00:00,NaN,2.0,2020-12-31 00:00:00,2021-03-07,ADALBERTORICARDODOSSANTOSFI93410690,ADALBERTORICARDODOSSANTOSFI93410690,ADALBERTORICARDODOSSANTOSFI92410690,ADALBERTORICARDODOSSANTOSFI94410690,ADALBERTORICARDODOSSANTOSFI21122020,ADALBERTORICARDODOSSANTOSFI31122020
890536,975719,-1.0,PR,410690,410690,ADAO SOARES SIQUEIRA,M,86,"COVID-19, Biologia Molecular",2021-03-02,2021-04-14 00:00:00,2021-03-02 00:00:00,2.0,2021-03-13 00:00:00,2021-04-14,ADAOSOARESSIQUEIRA86410690,ADAOSOARESSIQUEIRA86410690,ADAOSOARESSIQUEIRA85410690,ADAOSOARESSIQUEIRA87410690,ADAOSOARESSIQUEIRA02032021,ADAOSOARESSIQUEIRA13032021
285725,363532,-1.0,PR,410690,410690,ADELAIDE DAS CHAGAS LIMA,F,86,Vírus Respiratório,2020-11-29,2020-12-03 00:00:00,2020-11-29 00:00:00,2.0,2020-12-02 00:00:00,2020-12-03,ADELAIDEDASCHAGASLIMA86410690,ADELAIDEDASCHAGASLIMA86410690,ADELAIDEDASCHAGASLIMA85410690,ADELAIDEDASCHAGASLIMA87410690,ADELAIDEDASCHAGASLIMA29112020,ADELAIDEDASCHAGASLIMA02122020
329360,407186,-1.0,PR,410690,410690,ADELINA LEMOS,F,88,Vírus Respiratório,2020-07-31,2020-12-15 00:00:00,2020-07-28 00:00:00,2.0,2020-08-18 00:00:00,2020-12-15,ADELINALEMOS88410690,ADELINALEMOS88410690,ADELINALEMOS87410690,ADELINALEMOS89410690,ADELINALEMOS31072020,ADELINALEMOS18082020
976910,1062115,-1.0,PR,410690,411915,ADELIR RIBEIRO DA SILVA,M,50,"COVID-19, Biologia Molecular",2021-04-28,2021-05-08 00:00:00,NaN,2.0,2021-05-06 00:00:00,2021-05-08,ADELIRRIBEIRODASILVA50410690,ADELIRRIBEIRODASILVA50411915,ADELIRRIBEIRODASILVA49410690,ADELIRRIBEIRODASILVA51410690,ADELIRRIBEIRODASILVA28042021,ADELIRRIBEIRODASILVA06052021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023229,1108436,3655933.0,PR,410690,410690,WILSON RODRIGUES IANISCH,M,58,"COVID-19, Biologia Molecular",2021-05-15,2021-05-16 00:00:00,2021-05-11 00:00:00,2.0,1621900800000000000,2021-05-29,WILSONRODRIGUESIANISCH58410690,WILSONRODRIGUESIANISCH58410690,WILSONRODRIGUESIANISCH57410690,WILSONRODRIGUESIANISCH59410690,WILSONRODRIGUESIANISCH15052021,WILSONRODRIGUESIANISCH25052021
725539,807507,-1.0,PR,410690,410690,YVONE FELICISSIMA DE ANDRADE,F,96,"COVID-19, Biologia Molecular",2020-12-17,2021-03-07 00:00:00,NaN,2.0,2020-12-24 00:00:00,2021-03-07,YVONEFELICISSIMADEANDRADE96410690,YVONEFELICISSIMADEANDRADE96410690,YVONEFELICISSIMADEANDRADE95410690,YVONEFELICISSIMADEANDRADE97410690,YVONEFELICISSIMADEANDRADE17122020,YVONEFELICISSIMADEANDRADE24122020
246031,323797,-1.0,PR,410690,411915,ZAIRA VIEIRA DA SILVA,F,77,Teste Rápido IgM/IgG não discriminatório (Imun...,2020-11-18,2020-11-19 00:00:00,2020-11-15 00:00:00,2.0,2020-12-14 00:00:00,2021-01-28,ZAIRAVIEIRADASILVA77410690,ZAIRAVIEIRADASILVA77411915,ZAIRAVIEIRADASILVA76410690,ZAIRAVIEIRADASILVA78410690,ZAIRAVIEIRADASILVA18112020,ZAIRAVIEIRADASILVA14122020
55451,56741,-1.0,PR,410690,410420,ZEFERINO ALVES DA SILVA,M,82,LACEN,2020-07-19,2020-07-20 00:00:00,2020-07-09 00:00:00,2.0,2020-07-19 00:00:00,2020-07-19,ZEFERINOALVESDASILVA82410690,ZEFERINOALVESDASILVA82410420,ZEFERINOALVESDASILVA81410690,ZEFERINOALVESDASILVA83410690,ZEFERINOALVESDASILVA19072020,ZEFERINOALVESDASILVA19072020
